# Model Tester
<img src="https://www.teamly.com/blog/wp-content/uploads/2022/01/What-is-artificial-intelligence-in-project-management.png">

In [1]:
# to subpress warning
import warnings
warnings.filterwarnings("ignore")


# The OS module in Python provides functions for interacting with the operating system.
import os

# Matplotlib is a data visualization and graphical plotting library for Python.
import matplotlib.pyplot as plt

# seaborn is alse a data visualization and graphical plotting library for Python.
import seaborn as sn
import numpy as np
# used to display markdown,image,control (frontend utilities)
from IPython.display import display, clear_output
import pandas as pd
# computer vision library
import cv2
import pickle
import sys
import tensorflow as tf
import mediapipe as mp

import time

import random

In [2]:
PATH=r"D:/Image_datasets/asl_dataset"

In [3]:
class handDetector:
    def __init__(self, staticImageMode=False, maxNumHands=2, minDetectionConfidence=0.5, trackCon=0.5):
        self.results = None
        self.staticImageMode = staticImageMode
        self.maxNumberHands = maxNumHands
        self.minDetectionConfidence = minDetectionConfidence
        self.trackCon = trackCon
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(
            static_image_mode=self.staticImageMode,
            max_num_hands=self.maxNumberHands,
            min_detection_confidence=self.minDetectionConfidence,
            min_tracking_confidence=self.trackCon)

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)
        # for rect in self.results.hand_rects:
        #     print(rect)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mp_drawing.draw_landmarks(img, handLms,
                                                   self.mpHands.HAND_CONNECTIONS)
        return img

    def findPosition(self, img, maxHandNo=1, draw=False):
        mainlmlist = []
        handsType = []
        # handtype=[0,0]
        if self.results.multi_handedness:
            for hand in self.results.multi_handedness:
                # print(hand)
                # print(hand.classification)
                # print(hand.classification[0])
                handType = hand.classification[0].label
#                 print(handType)
                handsType.append(handType)

        # print(len(self.results.multi_hand_landmarks[0]))
        if self.results.multi_hand_landmarks:
            for myHand in self.results.multi_hand_landmarks:
                lmList = []
                if self.results.multi_hand_landmarks:

                    for pid, lm in enumerate(myHand.landmark):
                        # print(id, lm)
                        h, w, c = img.shape
                        cx, cy = int(lm.x * w), int(lm.y * h)
                        # print(id, cx, cy)
                        lmList.append([pid, cx, cy])
                        if draw:
                            cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
                mainlmlist.append(lmList)
        return mainlmlist, handsType,img



def predictSign(test,model):
#     print(test.shape)
    y_pred=model.predict(test)
    y_pred_labels=[unique_sign[np.argmax(i)] for i in y_pred]
    return y_pred_labels

In [4]:
def trackHand(handDetectorModel,DPath,relativePath):
    frame=cv2.imread(os.path.join(DPath,relativePath))
    frame=cv2.resize(frame, (400, 400))
    clone = handDetectorModel.findHands(frame.copy())
    mainlmList, handsType = handDetectorModel.findPosition(clone, draw=False)
    flattenedList=[]
#     print(clone)
#     print(mainlmList)
    if len(mainlmList)==0:
        return "empty",[]
    
    for keypoint in mainlmList[0]:
        flattenedList.append(keypoint[1])
        flattenedList.append(keypoint[2])
    return handsType[0],flattenedList


def Predict(relativePath,model,handDetectorModel,org):
    start_time = time.time()
    handType,pointslist=trackHand(handDetectorModel,PATH, relativePath)
    if handType=="empty":
        print("image : {} -> hand not found -> Aborted".format(relativePath))
        return None
    if handType=="Right":
        pointslist+=[1,0]
    else:
        pointslist+=[0,1]
    
    data=np.array(pointslist)
    df = pd.DataFrame([data])
    pred=predictSign(df,model)[0]
    print("image : {} | predicted {} | original : {}".format(relativePath,pred,str(org)))
    exeTime=time.time()-start_time
    print("Execution time :{}ms".format(round(exeTime*100,2)))
    return True

In [9]:
model = tf.keras.models.load_model('./model.h5')
unique_sign=[]
with (open("test_data.pkl", "rb")) as openfile:
        try:
            test_object=pickle.load(openfile)
        except EOFError as e:
            print("Error : ",e)
unique_sign=test_object["unique_sign"]
handDetectorModel=handDetector()
    
Predict(r"q/hand1_q_bot_seg_1_cropped.jpeg",model,handDetectorModel,"q")

image : q/hand1_q_bot_seg_1_cropped.jpeg -> hand not found -> Aborted


In [10]:
labeldirs=["q"]
right=0
wrong=0
skipped=0
for labeldir in labeldirs:
    for filepath in os.listdir(os.path.join(PATH,labeldir)):
        relativePath=labeldir+"/"+filepath
        feed=Predict(relativePath,model,handDetectorModel,labeldir)
        if feed==None:
            skipped+=1
            continue
        elif feed:
            right+=1
        else:
            wrong+=1

print("\n\n-----| Evaluation Result |-----")
print("Total : {} \nRight : {} \nWrong : {}".format(right+wrong,right,wrong))
print("Skipped :",skipped)
print("Accuracy : {}".format(right/(right+wrong)*100,2))
    

image : q/hand1_q_bot_seg_1_cropped.jpeg -> hand not found -> Aborted
image : q/hand1_q_bot_seg_2_cropped.jpeg -> hand not found -> Aborted
image : q/hand1_q_bot_seg_3_cropped.jpeg -> hand not found -> Aborted
image : q/hand1_q_bot_seg_4_cropped.jpeg -> hand not found -> Aborted
image : q/hand1_q_bot_seg_5_cropped.jpeg -> hand not found -> Aborted
image : q/hand1_q_dif_seg_1_cropped.jpeg -> hand not found -> Aborted
image : q/hand1_q_dif_seg_2_cropped.jpeg -> hand not found -> Aborted
image : q/hand1_q_dif_seg_3_cropped.jpeg -> hand not found -> Aborted
image : q/hand1_q_dif_seg_4_cropped.jpeg -> hand not found -> Aborted
image : q/hand1_q_dif_seg_5_cropped.jpeg -> hand not found -> Aborted
image : q/hand1_q_left_seg_1_cropped.jpeg -> hand not found -> Aborted
image : q/hand1_q_left_seg_2_cropped.jpeg -> hand not found -> Aborted
image : q/hand1_q_left_seg_3_cropped.jpeg -> hand not found -> Aborted
image : q/hand1_q_left_seg_4_cropped.jpeg -> hand not found -> Aborted
image : q/hand1_